In [1]:
library(limma)

In [2]:
# Charger le pseudo-bulk
pb <- read.csv("pseudobulk_matrix.csv")

# Séparer l’expression et les métadonnées
expr <- pb[, !(colnames(pb) %in% c("class", "donor_id", "disease"))]
expr <- as.matrix(expr)

meta <- data.frame(
  class = pb$class,
  donor = pb$donor_id,
  disease = factor(pb$disease)
)
rownames(expr) <- paste(pb$class, pb$donor_id, pb$disease, sep="_")


In [35]:
# Liste des cell-types
celltypes <- unique(meta$class)

# Création d’un dossier de sortie
dir.create("DE_results", showWarnings = FALSE)

meta$class <- gsub("[/ ]+", "_", meta$class)

for (ct in celltypes) {
  
  cat("\n=== Cell type :", ct, "===\n")
  
  # Sélection des donors de ce cell type
  idx <- meta$class == ct
  expr_ct <- expr[idx, ]
  meta_ct <- meta[idx, ]
  
  expr_ct <- expr_ct[, colMeans(expr_ct, na.rm=TRUE) > 0.01 & apply(expr_ct, 2, var) > 1e-5]


  print(table(meta_ct$disease))

  # Relevel pour que CTRL = reference
  meta_ct$disease <- factor(meta_ct$disease, levels = c("normal", "dementia || Alzheimer disease",
                                                       "dementia || Parkinson disease"))
  
  # Design matrix
  design <- model.matrix(~ 0 + disease, data=meta_ct)
  colnames(design) <- c("CTRL", "AD", "PD")
  
  cat("Design matrix OK\n")
  
  # modèle limma-trend
  fit <- lmFit(t(expr_ct), design)

  contrasts <- makeContrasts(
    AD_vs_CTRL = AD - CTRL,
    PD_vs_CTRL = PD - CTRL,
    AD_vs_PD   = AD - PD,
    levels = design
  )
  
  fit2 <- contrasts.fit(fit, contrasts)
  fit2 <- eBayes(fit2, trend = TRUE)
  
  # cell type nettoyé
  ct_clean <- gsub("[/ ]+", "_", ct)

# Export
  write.csv(topTable(fit2, coef="AD_vs_CTRL", number=Inf),
            paste0("DE_results/DE_", ct_clean, "_ADvsCTRL.csv"))
  
  write.csv(topTable(fit2, coef="PD_vs_CTRL", number=Inf),
            paste0("DE_results/DE_", ct_clean, "_PDvsCTRL.csv"))
  
  write.csv(topTable(fit2, coef="AD_vs_PD", number=Inf),
            paste0("DE_results/DE_", ct_clean, "_ADvsPD.csv"))
}


=== Cell type : Astro ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                             3 
                       normal 
                            9 
Design matrix OK

=== Cell type : EN ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                             3 
                       normal 
                            9 
Design matrix OK

=== Cell type : Endo ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                             3 
                       normal 
                            9 
Design matrix OK

=== Cell type : IN ===

dementia || Alzheimer disease dementia || Parkinson disease 
                           17                             3 
                       normal 
                            9 
Design matrix OK

=== Cell type : Immune ===

dementia || Alzheimer disease dementia || Parkinson disease 
 

In [36]:
dim(expr_ct)

[1]    29 17298

- aucun donneur n’a plusieurs diseases
- pas de duplication artificielle
- cellules présentes seulement pour certaines conditions
- distribution logique (tous les donneurs n’ont pas toutes les cell types)

Certains gènes ont une variance extrêmement faible entre donors dans un cell type donné.

C’est normal en pseudo-bulk log-normalisé
- encore plus normal quand :
- un cell type a peu de donors (ex : 2–3 en PD)
- les valeurs sont des moyennes log-normalisées
- beaucoup de gènes sont presque non exprimés

In [33]:
head(meta)
head(expr)
table(meta$class, meta$disease)
apply(expr, 1, sd)[1:20]  # 20 gènes au hasard

,class,donor,disease
,<chr>,<chr>,<fct>
1,Astro,Donor_31,dementia || Alzheimer disease
2,Astro,Donor_82,dementia || Alzheimer disease
3,Astro,Donor_137,dementia || Alzheimer disease
4,Astro,Donor_228,dementia || Alzheimer disease
5,Astro,Donor_310,dementia || Alzheimer disease
6,Astro,Donor_333,dementia || Alzheimer disease


,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,⋯,ENSG00000288600,ENSG00000288602,ENSG00000288605,ENSG00000288606,ENSG00000288611,ENSG00000288612,ENSG00000288622,ENSG00000288637,ENSG00000288642,ENSG00000288643
Astro_Donor_31_dementia || Alzheimer disease,0.16511628,0.016279070,0.5627907,0.4023256,0.32325580,0.002325581,0.009302326,0.14883721,0.8604651,0.1953488,⋯,0.000000000,0.22790697,0.08604651,0.01395349,0.000000000,0.04883721,0.009302326,0.006976744,0.03488372,0.03023256
Astro_Donor_82_dementia || Alzheimer disease,0.02547771,0.000000000,0.2675159,0.1464968,0.07643312,0.012738854,0.025477707,0.07006369,0.3503185,0.1337580,⋯,0.000000000,0.03821656,0.01273885,0.00000000,0.000000000,0.01910828,0.000000000,0.000000000,0.06369427,0.01273885
Astro_Donor_137_dementia || Alzheimer disease,0.28750000,0.007500000,0.4150000,0.3500000,0.21250000,0.002500000,0.007500000,0.11500000,1.1625000,0.2000000,⋯,0.005000000,0.31250000,0.07750000,0.01250000,0.000000000,0.05000000,0.010000000,0.005000000,0.04250000,0.04000000
Astro_Donor_228_dementia || Alzheimer disease,0.11130742,0.001766784,0.4558304,0.3480565,0.21731448,0.003533569,0.001766784,0.09717315,1.1130742,0.1766784,⋯,0.001766784,0.21024735,0.04416961,0.01766785,0.001766784,0.05300353,0.005300353,0.008833922,0.01766785,0.03003534
Astro_Donor_310_dementia || Alzheimer disease,0.08391608,0.002331002,0.4825175,0.2937063,0.22144522,0.002331002,0.009324009,0.13752913,0.7249417,0.1934732,⋯,0.000000000,0.23543124,0.07459208,0.01165501,0.000000000,0.04195804,0.000000000,0.013986014,0.03962704,0.02797203
Astro_Donor_333_dementia || Alzheimer disease,0.27638190,0.030150754,0.3517588,0.2562814,0.25376883,0.002512563,0.012562814,0.08793970,0.5527638,0.1658292,⋯,0.000000000,0.17336683,0.03266332,0.01758794,0.002512563,0.04522613,0.002512563,0.007537689,0.09798995,0.04020100


        
         dementia || Alzheimer disease dementia || Parkinson disease normal
  Astro                             17                             3      9
  EN                                17                             3      9
  Endo                              17                             3      9
  Immune                            17                             3      9
  IN                                17                             3      9
  Mural                             17                             3      9
  Oligo                             17                             3      9
  OPC                               17                             3      9

Astro_Donor_31_dementia || Alzheimer disease 
                                     4.178520 
 Astro_Donor_82_dementia || Alzheimer disease 
                                     2.322284 
Astro_Donor_137_dementia || Alzheimer disease 
                                     4.613824 
Astro_Donor_228_dementia || Alzheimer disease 
                                     3.175841 
Astro_Donor_310_dementia || Alzheimer disease 
                                     3.632972 
Astro_Donor_333_dementia || Alzheimer disease 
                                     5.030500 
                       Astro_Donor_478_normal 
                                     1.992191 
Astro_Donor_540_dementia || Alzheimer disease 
                                     7.403740 
Astro_Donor_545_dementia || Parkinson disease 
                                     2.081207 
                       Astro_Donor_601_normal 
                                     2.307823 
                       Astro_Donor_609_normal 
                                     2.418510 
Astro_Donor_614_dementia || Alzheimer disease 
                                     2.640386 
Astro_Donor_616_dementia || Alzheimer disease 
                                     2.552363 
                       Astro_Donor_634_normal 
                                     4.383488 
Astro_Donor_638_dementia || Alzheimer disease 
                                     1.986083 
                       Astro_Donor_721_normal 
                                     2.750514 
Astro_Donor_730_dementia || Alzheimer disease 
                                     1.068930 
                       Astro_Donor_735_normal 
                                     3.077410 
Astro_Donor_785_dementia || Alzheimer disease 
                                     1.346454 
Astro_Donor_865_dementia || Alzheimer disease 
                                     2.847305

In [40]:
expr_ct.var

ERROR: Error: object 'expr_ct.var' not found
